In [34]:
import os
import json
import pickle
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, LLMChain, StuffDocumentsChain
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.schema import Document

In [2]:
pdf_loader = DirectoryLoader(r"C:\Major_Project\NITT_ChatBot\data", glob="*.pdf", loader_cls=PyPDFLoader)
pdf_docs = pdf_loader.load()

In [3]:
print(pdf_docs)

[Document(metadata={'producer': 'Microsoft® Word LTSC', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2024-04-25T11:16:36+05:30', 'author': 'staff', 'moddate': '2024-04-25T11:16:36+05:30', 'source': 'C:\\Major_Project\\NITT_ChatBot\\data\\B.Arch 2024.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='ACADEMIC OFFICE / FEES SECTION \nNATIONAL INSTITUTE OF TECHNOLOGY \nTIRUCHIRAPPALLI - 620 015, TAMIL NADU, INDIA \n \nINSTITUTE FEES STRUCTURE FOR 2024 ADMISSION BATCH \n COURSE: B.Arch.     CATEGORY: OPEN/OPEN-EWS/OBC-NCL/ICCR/DASA (CIWG) \nSl.No. SEMESTER I II III IV V VI VII VIII IX X \n1 Tuition Fee * 62500 62500 62500 62500 62500 62500 62500 62500 62500 62500 \nAnnual Fee \n1 Computer Fee 2650 -- 2900 -- 3200 -- 3500 -- 3850 -- \n2 Internet Fee 1100 -- 1200 -- 1350 -- 1500 -- 1650 -- \n3 Library fee 4950 -- 5500 -- 6050 -- 6600 -- 7250 -- \n4 Examination fee 2650 -- 2900 -- 3200 -- 3500 -- 3850 -- \n5 Registration-Enrolment fee 550 -- 600 -- 650 -- 700 -- 750 -

In [4]:
web_loader = WebBaseLoader(["https://www.nitt.edu/"])
web_docs = web_loader.load()

In [5]:
print(web_docs[0].page_content)













National Institute of Technology, Trichy










































Placements

Library
Departments

Events

CC

Translate





















Students Webmail
Staff Webmail
Sitemap
Dashboard





View
 Login
Register
Search
 








Select Language



Afrikaans
Albanian
Amharic
Arabic
Armenian
Azerbaijani
Basque
Belarusian
Bengali
Bosnian

 Bulgarian
Catalan
Cebuano
Chichewa
Chinese (Simplified)
Chinese (Traditional)
Corsican
Croatian
Czech
Danish
Dutch

 

Esperanto
Estonian
Filipino
Finnish
French
Frisian
Galician
Georgian
German
Greek
Gujarati

 

Haitian Creole
Hausa
Hawaiian
Hebrew
Hindi
Hmong
Hungarian
Icelandic
Igbo
Indonesian
Irish

 
Italian
Japanese
Javanese
Kannada
Kazakh
Khmer
Korean
Kurdish (Kurmanji)
Kyrgyz
Lao
Latin

 
Latvian
Lithuanian
Luxembourgish
Macedonian
Malagasy
Malay
Malayalam
Maltese
Maori
Marathi
Mongolian

 
Myanmar (Burmese)
Nepali
Norwegian
Pashto
Persian
Polish
Portuguese
Punjabi
Romanian
Russian
Samoan

 
Scots Gaelic
Serbia

In [6]:
import bs4

In [7]:
# Home Page
web_loader1 = WebBaseLoader(web_paths=("https://www.nitt.edu/",),bs_kwargs=dict(parse_only=bs4.SoupStrainer(
    class_=("vc_rows wpb_rows vc_rows-fluid vc_custom_1439533085489","vc_rows wpb_rows vc_rows-fluid vc_custom_1439435173873","vc_rows wpb_rows vc_rows-fluid vc_custom_1439435173873","blink")
)))
web_docs_Home = web_loader1.load()

In [8]:
print(web_docs_Home[0].page_content)


NIRF 2024 - First among NITs, Ninth in Engineering....
Engagement of Apprentice Trainees for the year 2024-2025 - Shortlisted Candidates....
First Schedule....
Second Schedule....
RECAL ScholaIrship 2024 Application Form, last date extended till January 21, 2025....
Congratulations to our B.Tech Students for receiving prestigious Mitacs Globalink Research Internship 2024....
Certificate of Appreciation - NIPAM 2022....F
NIT-T appointed as the National MOOCs Coordinator for offering Engineering Courses at UG & PG level by MHRD, GOI....
NIT, Tiruchirapalli gets Prime Minister's Research Fellowship (PMRF) Scheme ....
For skill hub courses and applications https://skillhub.nitt.edu/ ....
NIT-T Ambulance 9486001162 ... NIT-T Hospital 91-431-2503860... NIT-T Security Office 9486001168 / 91-431-2503900...
Anti-Ragging Helpline Mobile Number: 9486001180 and emailD: antiragging@nitt.edu....
Internal Complaints committee to address Sexual Harassment of Women at Workplace (Prevention, Prohibitio

In [9]:
# About
web_loader2 = WebBaseLoader(web_paths=("https://www.nitt.edu/home/about/","https://www.nitt.edu/home/about/nitt/","https://www.nitt.edu/home/about/trichy/","https://www.nitt.edu/home/about/ataglance/"),bs_kwargs=dict(parse_only=bs4.SoupStrainer(
    id=("side")
)))
web_docs_about = web_loader2.load()

In [10]:
print(web_docs_about[0].page_content)
print(web_docs_about[1].page_content)
print(web_docs_about[2].page_content)
print(web_docs_about[3].page_content)




About

Vision, Mission, Core Values and Goals
NIT Tiruchirappalli, through its Vision, Mission and Core Values, defines herself as: 
An Indian institution with world standards
A global pool of talented students, committed faculty and conscientious researchers
Responsive to real-world problems and, through a synergy of education and research, engineer a better society
VISION

To be a university globally trusted for technical excellence where learning and research integrate to sustain society and industry.

MISSION

To offer undergraduate, postgraduate, doctoral and modular programmes in multi-disciplinary / inter-disciplinary and emerging areas.
To create a converging learning environment to serve a dynamically evolving society.
To promote innovation for sustainable solutions by forging global collaborations with academia and industry in cutting-edge research.
To be an intellectual ecosystem where human capabilities can develop holistically.

Core Values:
Integrity
Honest in intenti

In [11]:
# Academic
web_loader3 = WebBaseLoader(web_paths=("https://www.nitt.edu/home/academics/","https://www.nitt.edu/home/academics/contact_details/","https://www.nitt.edu/home/academics/programmes/","https://www.nitt.edu/home/academics/office/","https://www.nitt.edu/home/academics/admission_procedure/","https://www.nitt.edu/home/academics/anti_ragging/"),bs_kwargs=dict(parse_only=bs4.SoupStrainer(
    id=("side")
)))
web_docs_academic = web_loader3.load()

In [12]:
print(web_docs_academic[5].page_content)




Anti ragging
Ragging is a disturbing reality in the higher education system of our country. Despite the fact that over the years, ragging has claimed hundreds of innocent lives and has ruined the careers of thousands of bright students, the practice is still perceived by many as a way of ‘familiarisation’ and an ‘initiation into the real world’ for young college-going students. The Ragging is defined as any disorderly conduct, whether by words spoken or written or by an act, has the effect of teasing, treating, or handling with rudeness a fresher or a junior student. Indulging in a rowdy or undisciplined activity that causes or is likely to cause annoyance, hardship, or psychological harm or to raise fear or apprehension thereof in a fresher or junior student. Asking the students to do any act or perform something that such students will not do in the ordinary course and which has the effect of causing or generating a sense of shame or embarrassment so as to adversely affect the phy

In [13]:
#Administration
web_loader4 = WebBaseLoader(web_paths=("https://www.nitt.edu/home/administration/","https://www.nitt.edu/home/administration/director/","https://www.nitt.edu/home/administration/deans/","https://www.nitt.edu/home/administration/hods/"),bs_kwargs=dict(parse_only=bs4.SoupStrainer(
    id=("side")
)))
web_docs_admins = web_loader4.load()

In [14]:
print(web_docs_admins[0].page_content)
print(web_docs_admins[1].page_content)
print(web_docs_admins[2].page_content)
print(web_docs_admins[3].page_content)




Administration

The NITs were carved out of 17 Regional Engineering Colleges across India and are now the fully funded institutions under the Central Government and declared as an Institute of National importance under the NATIONAL INSTITUTES OF TECHNOLOGY ACT, 2007. The move was intended to make the institutions centers of excellence and being developed as autonomous and flexible academic institutions of excellence to meet the sweeping changes taking place in the industrial environment in post liberalized India and also the rapidly changing scene of technical education globally. NIT Tiruchirappalli is one among the 31 NITs and its basic structure of organisation, functions and powers of the Institute are briefed in the NIT Acts & Statutes. A large number of rules, regulations, ordinances, policy decisions etc. have been formulated by the Board of Governors, Senate and other authorities of the Institute for regulating the day-to-day work of the expanded activities of the Institute.


In [15]:
#Admission
web_loader5 = WebBaseLoader(web_paths=("https://www.nitt.edu/home/admissions/btech/","https://www.nitt.edu/home/admissions/mtech/","https://www.nitt.edu/home/admissions/msc/","https://www.nitt.edu/home/admissions/mca/","https://www.nitt.edu/home/admissions/mba/",
                                       "https://www.nitt.edu/home/admissions/ma/","https://www.nitt.edu/home/admissions/phd/","https://www.nitt.edu/home/admissions/bscbed/"),bs_kwargs=dict(parse_only=bs4.SoupStrainer(
    id=("side")
)))
web_docs_admission = web_loader5.load()

In [16]:
print(web_docs_admission[0].page_content)
print(web_docs_admission[1].page_content)
print(web_docs_admission[2].page_content)
print(web_docs_admission[3].page_content)
print(web_docs_admission[4].page_content)
print(web_docs_admission[5].page_content)
print(web_docs_admission[6].page_content)
print(web_docs_admission[7].page_content)




B. Tech. / B. Arch.

Orientation Programme
The Orientation Programme for first-year B.Tech. and B.Arch. students is a pivotal initiative designed to ease the transition from school to college life, setting the stage for a successful academic journey. This program typically includes an introduction to the institution's ethos, academic expectations, and available resources. Students are familiarized with the curriculum, departmental facilities, and campus infrastructure. Additionally, sessions are organized on time management, study techniques, and the importance of extracurricular activities, fostering a well-rounded development. The program also provides a platform for students to interact with faculty, senior students, and peers, creating a sense of community and belonging from the outset. Through this holistic approach, the Orientation Programme aims to equip students with the knowledge, skills, and confidence to navigate their first year with ease.

Invitation_Orientation Program

In [17]:
#Student Welfare
web_loader6 = WebBaseLoader(web_paths=("https://www.nitt.edu/home/students/studentcouncil/",),bs_kwargs=dict(parse_only=bs4.SoupStrainer(
    id=("side")
)))
web_docs_SW = web_loader6.load()

In [18]:
print(web_docs_SW[0].page_content)




Student Council
Office Bearers AY 2024-25





Akshat Kedia
President
Email: studentpresident@nitt.edu
Phone: 9486001186 (CUG); 9771006409





Karumuri Sri Harshitha
Vice-President
Phone: 9489066201 (CUG); 7207325892





Lakshmikanth Pola
General Secretary
Phone: 9361940296





Aastha Agarwal
General Secretary (Girls)
Phone: 9155482609





Charan N
Joint-Secretary
Phone: 9150660402





Jeevanaa Suresh Babu
Joint-Secretray (Girls)
Phone: 7200710109



 

Manish Choudhary
PG Secretary
Phone: 8789585894





Bireswar Das
PhD Secretary
Phone: 9348051447




Contact: Council Room (Admin building)
Email: studentcouncil@nitt.edu
Election Details AY 2024-25

Election Report AY 2024-25
Election Results AY 2024-25
Presiding and Polling Officers for Students' Council Election AY 2024-25
Work Flow for Students' Council Election AY 2024-25
Final List of Candidates for the Students' Council Election AY 2024-25
List of Eligible Candidates for the Students' Council Election AY 2024-25
Students

In [19]:
#Event
web_loader7 = WebBaseLoader(web_paths=("https://www.nitt.edu/home/students/events/","https://www.nitt.edu/home/students/events/pragyan/","https://www.nitt.edu/home/students/events/festember/","https://www.nitt.edu/home/students/events/nittfest/","https://www.nitt.edu/home/students/events/symposiums/","https://www.nitt.edu/home/academics/iday/"),bs_kwargs=dict(parse_only=bs4.SoupStrainer(
    id=("side")
)))
web_docs_Event= web_loader7.load()

In [20]:
for i in web_docs_Event:
    print(i.page_content)




Events
The College calendar is interspersed with numerous events whether technical or cultural, at inter departmental level or inter college level. The organisation and execution of most of these events is handled by students themselves guided by the faculty incharge. Here is a brief overview of the various events that form a part of the annual lifestyle of a NITTian.
 

Convocation Day
Alumni Day
Pragyan - Technical Festival 
Festember - Cultural Festival
NITTFEST - Inter Departmental Cultural Festival
Golden Jubilee
Student Technical Symposiums
Sports Day
Completed Events
Media Monitor
Communitty

 
 

 
 

Media Monitor
Press Reports / Report on Events
community - Newsletter






Pragyan
Since its inception in 2005, Pragyan, NIT Trichy’s annual international techno-managerial festival, has provided a platform to foster innovative thinking and technical acumen among the bright and developing minds of India. In its short yet impactful run, Pragyan has enjoyed footfall from over 10

In [21]:
# Facilities and Services

web_loader8 = WebBaseLoader(web_paths=("https://www.nitt.edu/home/students/facilitiesnservices/","https://www.nitt.edu/home/students/facilitiesnservices/ComputerSupportGroup/","https://www.nitt.edu/home/students/facilitiesnservices/tp/","https://www.nitt.edu/home/students/facilitiesnservices/tp/procedure/","https://www.nitt.edu/home/students/facilitiesnservices/tp/guidance/","https://www.nitt.edu/home/students/facilitiesnservices/library/"
                                       ,"https://www.nitt.edu/home/students/facilitiesnservices/hostelsnmess/hostels/","https://www.nitt.edu/home/students/facilitiesnservices/hostelsnmess/messes/","https://www.nitt.edu/home/students/facilitiesnservices/hospital/","https://www.nitt.edu/home/students/facilitiesnservices/transport/","https://www.nitt.edu/home/students/placestovisit/"),bs_kwargs=dict(parse_only=bs4.SoupStrainer(
    id=("side")
)))
web_docs_FS=web_loader8.load()

In [22]:
for i in web_docs_FS:
    print(i.page_content)




Facilities and Services
The facilities and services that NITT renders to the students,faculty and staff members make it stand among the world class institutes. The top class amenities provided to the students makes the stay in NITT a memorable experience.The presence of modern facilities furthers the overall development of all the residents of NITT and also assist students and faculty members to work in a co-operative way enhancing the academic growth of NITTians.

The Octagon - Computer Center, Computer Support Group
Param Porul supercomputer
Training and Placement
Library
Hostels and Mess
Hospital
Central Workshop
Shopping Center
Transport
Physical Education / Sports Center
Swimming Pool
Security
Campus Communication Services
Estate Maintenance
Lecture Hall Complex
First year co-ordinator office
Guest House 
Quarters






Computer Support Group

The Octagon - Main CC building 
The OCTAGON - Computer Center is the sterling hallmark of this campus was opened with the theme of "Dare

In [23]:
# CA Department 

web_loader9 = WebBaseLoader(web_paths=("https://www.nitt.edu/home/academics/departments/ca/","https://www.nitt.edu/home/academics/departments/ca/vision/","https://www.nitt.edu/home/academics/departments/ca/programmes/mca/",
                                       "https://www.nitt.edu/home/academics/departments/ca/programmes/msc_computerscience/","https://www.nitt.edu/home/academics/departments/ca/programmes/mtech/","https://www.nitt.edu/home/academics/departments/ca/facultymca/","https://www.nitt.edu/home/academics/departments/ca/staffs/",
                                       "https://www.nitt.edu/home/academics/departments/ca/associations/"),bs_kwargs=dict(parse_only=bs4.SoupStrainer(
    id=("side")
)))
web_docs_CA =web_loader9.load()

In [24]:
for i in web_docs_CA:
    print(i.page_content)




Computer Applications
                                            MCA Programme is NBA accredited for 5 years from 2017-18 to 2021-22 
Welcome to Department of Computer Applications 
 
The Department of Computer Applications is one of the pioneering departments of the institution that offers Information Technology courses such as MCA and one among the top five offering MCA courses in the country. It is committed to impart quality education in the sub-fields of IT, a field growing in leaps and bounds. The curriculum is so made that the course provides a good theoretical foundation through high-quality teaching complemented by extensive practical training. It is dedicated to the mission of inculcating value-based, socially committed professionalism to the cause of overall development of students and society
 
This department currently offers three full-time post-graduate programmes — three-year Master of Computer Applications (MCA), two-year Master of Science in Computer Science and t

In [25]:
web_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
pdf_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
pdf_chunks = pdf_splitter.split_documents(pdf_docs)
web_chunks = web_splitter.split_documents(web_docs_Home)+web_splitter.split_documents(web_docs_about)+web_splitter.split_documents(web_docs_admins)+web_splitter.split_documents(web_docs_admission)+web_splitter.split_documents(web_docs_academic)+ web_splitter.split_documents(web_docs_SW)+web_splitter.split_documents(web_docs_CA)+web_splitter.split_documents(web_docs_Event)+web_splitter.split_documents(web_docs_FS)
all_chunks = pdf_chunks + web_chunks

In [26]:
print(pdf_chunks)

[Document(metadata={'producer': 'Microsoft® Word LTSC', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2024-04-25T11:16:36+05:30', 'author': 'staff', 'moddate': '2024-04-25T11:16:36+05:30', 'source': 'C:\\Major_Project\\NITT_ChatBot\\data\\B.Arch 2024.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='ACADEMIC OFFICE / FEES SECTION \nNATIONAL INSTITUTE OF TECHNOLOGY \nTIRUCHIRAPPALLI - 620 015, TAMIL NADU, INDIA \n \nINSTITUTE FEES STRUCTURE FOR 2024 ADMISSION BATCH \n COURSE: B.Arch.     CATEGORY: OPEN/OPEN-EWS/OBC-NCL/ICCR/DASA (CIWG) \nSl.No. SEMESTER I II III IV V VI VII VIII IX X \n1 Tuition Fee * 62500 62500 62500 62500 62500 62500 62500 62500 62500 62500 \nAnnual Fee \n1 Computer Fee 2650 -- 2900 -- 3200 -- 3500 -- 3850 -- \n2 Internet Fee 1100 -- 1200 -- 1350 -- 1500 -- 1650 -- \n3 Library fee 4950 -- 5500 -- 6050 -- 6600 -- 7250 -- \n4 Examination fee 2650 -- 2900 -- 3200 -- 3500 -- 3850 -- \n5 Registration-Enrolment fee 550 -- 600 -- 650 -- 700 -- 750 -

In [27]:
print(web_chunks)

[Document(metadata={'source': 'https://www.nitt.edu/'}, page_content='NIRF 2024 - First among NITs, Ninth in Engineering....\r\nEngagement of Apprentice Trainees for the year 2024-2025 - Shortlisted Candidates....\r\nFirst Schedule....\r\nSecond Schedule....\r\nRECAL ScholaIrship 2024 Application Form, last date extended till January 21, 2025....\r\nCongratulations to our B.Tech Students for receiving prestigious Mitacs Globalink Research Internship 2024....\r\nCertificate of Appreciation - NIPAM 2022....F'), Document(metadata={'source': 'https://www.nitt.edu/'}, page_content="Certificate of Appreciation - NIPAM 2022....F\r\nNIT-T appointed as the National MOOCs Coordinator for offering Engineering Courses at UG & PG level by MHRD, GOI....\r\nNIT, Tiruchirapalli gets Prime Minister's Research Fellowship (PMRF) Scheme ....\r\nFor skill hub courses and applications https://skillhub.nitt.edu/ ....\r\nNIT-T Ambulance 9486001162 ... NIT-T Hospital 91-431-2503860... NIT-T Security Office 948

In [28]:
with open(r"C:\Major_Project\NITT_ChatBot\data\FAQs.json", "r") as file:
    faqs = json.load(file)

# Convert FAQs into Documents


faq_docs = [
    Document(
        page_content=f"Question: {faq['patterns'][0]}\nAnswer: {faq['responses'][0]}",
        metadata={"tag": faq["tag"]}
    )
    for faq in faqs
]

In [29]:
print(faq_docs)

[Document(metadata={'tag': 'greeting'}, page_content='Question: Hi\nAnswer: Hello!'), Document(metadata={'tag': 'farewell'}, page_content='Question: Goodbye\nAnswer: Sad to see you go :('), Document(metadata={'tag': 'creator'}, page_content='Question: Who created you?\nAnswer: I was created by Sweety Mahale for the final Year college project.'), Document(metadata={'tag': 'identity'}, page_content="Question: What is your name?\nAnswer: I'm a NITT Chatbot. You can ask me anything about NIT trichy. I will provide you information about NIT Trichy."), Document(metadata={'tag': 'communication'}, page_content='Question: How can I communicate with the university or contact my ward?\nAnswer: You can communicate with the university through our official email, helpline number, or by visiting the respective department. For parents, we provide regular updates through email and dedicated portals.'), Document(metadata={'tag': 'casual_greeting'}, page_content="Question: What's up?\nAnswer: I'm here to

In [42]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Store PDFs & Website Data in FAISS
vector_db = FAISS.from_documents(all_chunks, embeddings)

# Store FAQs in FAISS
faq_vector_db = FAISS.from_documents(faq_docs, embeddings)

# Save FAISS indices manually
vector_db.save_local("faiss_index")

faq_vector_db.save_local("faq_index")

# Create Retrievers
retriever = vector_db.as_retriever(search_type="mmr")
faq_retriever = faq_vector_db.as_retriever(search_type="mmr")

In [43]:
os.environ["GROQ_API_KEY"] = "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"
llm = ChatGroq(model="llama-3.3-70b-versatile",temperature=0.2, max_tokens=800, streaming=True)

# Define Prompt for Structured Responses
prompt_template = PromptTemplate.from_template("""
You are a helpful AI assistant for NIT Trichy. Your job is to provide **accurate, concise, and structured** answers based on FAQs and retrieved documents.
                                               
### FAQs Context:
{faq_context}

### Retrieved Documents:
{document_context}

### User Question:
{question}

### Answer:
""")


llm_chain = LLMChain(llm=llm, prompt=prompt_template)

In [44]:
def retrieve_faq(user_question, k=3):
    """Retrieve the top-k most relevant FAQs based on similarity search."""
    similar_faqs = faq_retriever.get_relevant_documents(user_question, search_kwargs={"k": k})
    
    if similar_faqs:
        return "\n\n".join([faq.page_content for faq in similar_faqs])  # Combine FAQs
    return "No relevant FAQ found."  # Default response

def chatbot_response(user_question):
    """Integrate FAQs + document retrieval + LLM for structured responses."""
    retrieved_faq = retrieve_faq(user_question)
    retrieved_docs = retriever.get_relevant_documents(user_question)

    structured_prompt = prompt_template.format(
        faq_context=retrieved_faq,
        document_context="\n\n".join([doc.page_content for doc in retrieved_docs]),
        question=user_question
    )

    response = llm_chain.invoke({
    "faq_context": retrieved_faq if retrieved_faq else "No relevant FAQ found.",
    "document_context": "\n\n".join([doc.page_content for doc in retrieved_docs]) if retrieved_docs else "No relevant document found.",
    "question": user_question
})

    return response

In [41]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Chatbot: Goodbye!")
        break
    bot_response = chatbot_response(user_input)
    print("You:",user_input)
    print("Chatbot:", bot_response["text"])

C:\Users\Sweety\AppData\Local\Temp\ipykernel_5596\2095468567.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  similar_faqs = faq_retriever.get_relevant_documents(user_question, search_kwargs={"k": k})


You: HIii
Chatbot: Hello!
You: Tell me about nit trichy
Chatbot: **Introduction to NIT Trichy**

NIT Trichy, also known as National Institute of Technology Tiruchirappalli, is located in Tiruchirappalli, Tamil Nadu. The college is situated on the Tanjore Main Road, in National Highway 83, between BHEL and Thuvakudi.

**Key Highlights:**

1. **Location**: Tanjore Main Road, NH 83, between BHEL and Thuvakudi, Tiruchirappalli, TN.
2. **Admission Process**: Competitive admissions through JEE (Main) for undergraduate courses and GATE for post-graduate programs.
3. **Alumni Network**: Active and engaged alumni network with opportunities for students to connect through events, mentorship programs, and career panels.
4. **Infrastructure and Activities**: The institute offers a range of co-curricular and extra-curricular activities, including 'NITTFEST', an annual fine arts extravaganza.
5. **Connectivity**: Tiruchirappalli is well-connected by road, rail, and air to major cities in South India

In [ ]:
print(vector_db._collection.count())


1121
